In [ ]:
%load_ext autoreload

In [ ]:
%run ./common_init.ipynb

In [ ]:
%autoreload 2
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import HashingEncoder, OneHotEncoder, OrdinalEncoder

# Load custom code
import kdd98.data_handler as dh
import kdd98.utils_transformer as ut
from kdd98.transformers import *
from kdd98.config import Config

In [ ]:
# Where to save the figures
IMAGES_PATH = pathlib.Path(figure_output/'preprocessing')

pathlib.Path(IMAGES_PATH).mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = pathlib.Path(IMAGES_PATH/fig_id + "." + fig_extension)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Splitting into training- and test dataset

Before applying *any* transformations, the dataset will be split 80/20 into a learning and test set.

Let's look at feature TARGET_B, which describes whether a person has donated or not:

In [ ]:
learning_raw.TARGET_B.value_counts(normalize=True) # 5 % of recipients have donated.

We want to preserve this ratio in the split datasets. scikit-learn provides a method for achieving this.

In [ ]:
seed = Config.get.config("random_seed")
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size=0.8, random_state=seed)
for learn_index, test_index in splitter.split(learning_raw, learning_raw.TARGET_B.astype('int')):
    l_i = learn_index
    t_i = test_index
    kdd_learn = learning_raw.iloc[learn_index]
    kdd_test = learning_raw.iloc[test_index]

Now, check that the two sets are really disjoint

In [ ]:
set(kdd_learn.index).intersection(kdd_test.index)

Check the frequencies of the donors in the sets:

In [ ]:
kdd_learn['TARGET_B'].value_counts(normalize=True)

In [ ]:
kdd_test['TARGET_B'].value_counts(normalize=True)

## Separating features and label

First, we separate the features from the labels. We will also remove the label "TARGET_B", which is an indicator variable for donors that is no longer of interest

**All preprocessing is performed on *kdd_learn_feat***

In [ ]:
kdd_learn_feat = kdd_learn.drop(['TARGET_B', 'TARGET_D'],axis=1).copy()
kdd_learn_labels = kdd_learn[['TARGET_B','TARGET_D']].copy()

# Feature Selection
Meant to reduce dimensionality by selecting only features that are 'interesting enough' to be considered in order to boost performance of calculations / improve accuracy of the estimator
- By variance threshold
- Recursive Feature Elimination by Cross-Validation
- L1-based feature selection (Logistic Regression, Lasso, SVM)
- Tree-based feature selection

See [scikit-learn: feature selection](http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection)